In [1]:
from uatu.watchers import *
from sklearn.model_selection import train_test_split
%load_ext autoreload
%autoreload 2

/home/sean/.local/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
X,Y = get_xy_from_dir('../data/Box000/', 0)

In [3]:
train_dset = Dataset(X[:6], Y[:6], 2, shuffle = True, augment = True)
test_dset = Dataset(X[6:8], Y[6:8], 2, shuffle = True, augment = True)

data = (train_dset, test_dset, None)

In [4]:
device = 'CPU:0'

In [5]:
X.shape

(64, 64, 64, 64, 1)

In [6]:
def bayes_train(model_init_fn, optimizer_init_fn,data, device, fname, \
                restore = False,num_epochs = 1, print_every = 10, lr = 0.0005):
    tf.reset_default_graph()
    train_dset, val_dset, _ = data
    with tf.device(device):

        x = tf.placeholder(tf.float32, [None, 64,64,64,1])
        y = tf.placeholder(tf.float32, [None,2])

        training = tf.placeholder(tf.bool, name='training')

        preds = model_init_fn(x, training)
        
        log_s1 = tf.slice(preds , [0,2] , [-1,1] )
        log_s2 = tf.slice(preds , [0,3] , [-1,1] )
        rho = tf.tanh(tf.slice(preds, [0,4], [-1,1]))
        mu1 = tf.slice(preds , [0,0] , [-1,1] )
        mu2 = tf.slice(preds, [0,1], [-1,1]) 
        # avoiding building a matrix...
        
        z = tf.pow( mu1 -  tf.slice(y,[0,0], [-1,1]) , 2.)*tf.exp(-log_s1) +\
            tf.pow( mu2 -  tf.slice(y,[0,1], [-1,1]) , 2.)*tf.exp(-log_s2) -\
            2*rho*(mu1 - tf.slice(y,[0,0], [-1,1]))*(mu2 - tf.slice(y,[0,1], [-1,1]))*tf.sqrt(tf.exp(-log_s1)*tf.exp(-log_s2))
        
        loss = tf.reduce_mean(z/(2*(1-tf.pow(rho, 2.))) + 2*np.pi*tf.sqrt(tf.exp(-log_s1)*tf.exp(-log_s2)*(1-tf.pow(rho,2.))))
        
        #loss = tf.losses.absolute_difference(labels=y, predictions=preds, reduction=tf.losses.Reduction.SUM)
        
        #loss = tf.losses.mean_squared_error(labels=y, predictions=preds, reduction=tf.losses.Reduction.SUM)

        #loss = tf.reduce_mean(loss)

        optimizer = optimizer_init_fn(lr)
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_op = optimizer.minimize(loss)
        
    with tf.device('/cpu:0'):

        saver = tf.train.Saver()

    with tf.Session() as sess:
        if restore:
            saver.restore(sess, fname)
        else:
            sess.run(tf.global_variables_initializer())
        t = 0
        for epoch in xrange(num_epochs):
            print 'Starting epoch %d' % epoch
            for x_np, y_np in train_dset:
                feed_dict = {x: x_np, y: y_np, training: True}
                #loss_np, update_ops_np = sess.run([loss,update_ops], feed_dict=feed_dict)
                loss_np, _  = sess.run([loss, train_op], feed_dict=feed_dict)
                #out = sess.run([preds], feed_dict=feed_dict)
                
                if t % print_every == 0:
                    print 'Iteration %d, loss = %.4f' % (t, loss_np)
                    check_accuracy(sess, val_dset, x, preds, training=training)
                    print()
                    saver.save(sess, fname, global_step = t)
                
                t += 1

        saver.save(sess, fname, global_step = t) #save one last time

In [7]:
bayes_train(shallow_bayesian_convnet_init_fn, standard_optimizer_init_fn,data, device, fname = './shallow_bayes_test', num_epochs = 100, print_every = 1)

Starting epoch 0
Iteration 0, loss = 1587.3057
chi2: 37150187677.658
()


KeyboardInterrupt: 